In [1]:
# Imports
import numpy as np
import torch
import matplotlib.pyplot as plt

In [2]:
# Training parameters
learning_rate = 0.01
batch_size = 128
epochs = 100000

# Network parameters
image_dim = 28 * 28
gen_hidd_dim = 256
disc_hidd_dim = 256
z_noise_dim = 100 # Noise data points